In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

# Load dataset
file_path = "D:\\Vit Bhopal\\Year 3 sem 8 Summer\\epics\\backend\\Sheet 1-1-crop_yield_dataset.csv"
df = pd.read_csv(file_path)

# Encode categorical features
soil_encoder = LabelEncoder()
df["Soil_Type"] = soil_encoder.fit_transform(df["Soil_Type"])

crop_encoder = LabelEncoder()
df["Crop_Type"] = crop_encoder.fit_transform(df["Crop_Type"])

# Feature engineering
df["N_P_Ratio"] = df["N"] / (df["P"] + 1e-6)
df["N_K_Ratio"] = df["N"] / (df["K"] + 1e-6)

# Prepare features and target
X = df.drop(columns=["Date", "Crop_Yield", "Crop_Type", "Humidity"])
y = df["Crop_Type"]
y_yield = df["Crop_Yield"]

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split data
X_train, X_test, y_train, y_test, y_yield_train, y_yield_test = train_test_split(X_scaled, y, y_yield, test_size=0.2, random_state=42)

# Build ANN model
model = keras.Sequential([
    keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(len(np.unique(y)), activation='softmax')
])

# Compile model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

# Evaluate model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"ANN Crop Prediction Accuracy: {accuracy}")

# Build ANN model for yield prediction
yield_model = keras.Sequential([
    keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1, activation='linear')
])

# Compile yield model
yield_model.compile(optimizer='adam', loss='mse')

# Train yield model
yield_model.fit(X_train, y_yield_train, epochs=50, batch_size=32, validation_data=(X_test, y_yield_test))

# User input function
def get_user_input():
    soil_type = int(input("Enter Soil Type (Encoded value): "))
    soil_pH = float(input("Enter Soil pH: "))
    temperature = float(input("Enter Temperature: "))
    nitrogen = float(input("Enter Nitrogen (N) level: "))
    phosphorus = float(input("Enter Phosphorus (P) level: "))
    potassium = float(input("Enter Potassium (K) level: "))
    soil_quality = float(input("Enter Soil Quality: "))
    
    return pd.DataFrame([[soil_type, soil_pH, temperature, nitrogen, phosphorus, potassium, soil_quality,
                          nitrogen / (phosphorus + 1e-6), nitrogen / (potassium + 1e-6)]],
                        columns=["Soil_Type", "Soil_pH", "Temperature", "N", "P", "K", "Soil_Quality", 
                                 "N_P_Ratio", "N_K_Ratio"])

user_input = get_user_input()
user_input_scaled = scaler.transform(user_input)
crop_pred = np.argmax(model.predict(user_input_scaled), axis=-1)[0]
predicted_crop = crop_encoder.inverse_transform([crop_pred])[0]
yield_pred = yield_model.predict(user_input_scaled)[0][0]
print(f"Predicted Crop Type: {predicted_crop}")
print(f"Predicted Crop Yield: {yield_pred}")

model.save("crop_model.keras")   # Saves in TensorFlow format
yield_model.save("yield_model.keras")


C:\Users\User\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
913/913 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.6959 - loss: 0.8724 - val_accuracy: 0.8979 - val_loss: 0.1475
Epoch 2/50
913/913 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9005 - loss: 0.1456 - val_accuracy: 0.8988 - val_loss: 0.1434
Epoch 3/50
913/913 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9021 - loss: 0.1441 - val_accuracy: 0.9002 - val_loss: 0.1427
Epoch 4/50
913/913 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9003 - loss: 0.1408 - val_accuracy: 0.8973 - val_loss: 0.1441
Epoch 5/50
913/913 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8976 - loss: 0.1433 - val_accuracy: 0.8985 - val_loss: 0.1422
Epoch 6/50
913/913 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9013 - loss: 0.1387 - val_accuracy: 0.8983 - val_loss: 0.1435
Epoch 7/50
913/913 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9054 - loss: 0.1391 - val_accuracy: 0.8981 - val_loss: 0.1421
Epoch 8/50
913/913 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9001 - loss: 0.1381 - val_accuracy: 0.

ValueError: invalid literal for int() with base 10: 'Sandy'